In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4398k      0  0:00:18  0:00:17  0:00:01 5510k2M   69 55.4M    0     0  4094k      0  0:00:20  0:00:13  0:00:07 5411k    0  4444k      0  0:00:18  0:00:18 --:--:-- 5495k


In [9]:
!tar -xf aclImdb_v1.tar.gz

In [ ]:
from utils import create_dataloaders
from torch import optim
import torch
import torch.nn as nn

In [2]:
from transformer import Transformer

In [3]:
train_dir = "aclImdb/train"
train_dataloader, vocab = create_dataloaders(train_dir)

In [11]:
test_dir = "aclImdb/test"
test_dataloader, vocab = create_dataloaders(test_dir)

In [4]:
embed_dim = 128
num_heads = 2
dense_dim = 32

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
transformer = Transformer(embed_dim, dense_dim, num_heads, 20000).to(device)

rmsprop = optim.RMSprop(params=transformer.parameters(), lr=0.0001)
criterion = nn.BCELoss()

/home/ivan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
for epoch in range(10):
    transformer.train()
    correct_predictions = 0
    total_predictions = 0


    for batch in train_dataloader:
        text, label = batch

        rmsprop.zero_grad()

        output = transformer(text.to(device))

        loss = criterion(output[:, 0], label.to(device).float())

        correct_predictions += (output[:, 0] > 0.5).eq(label.to(device)).sum().item()
        total_predictions += len(label)


        loss.backward()
        rmsprop.step()


    print(f"Epoch: {epoch+1}, Loss: {loss.item()}, Accuracy: {correct_predictions / total_predictions * 100}")


Epoch: 1, Loss: 0.755528450012207, Accuracy: 65.704
Epoch: 2, Loss: 0.41941702365875244, Accuracy: 76.40400000000001
Epoch: 3, Loss: 0.7098008990287781, Accuracy: 79.264
Epoch: 4, Loss: 0.21520844101905823, Accuracy: 80.92399999999999
Epoch: 5, Loss: 0.24022485315799713, Accuracy: 82.3
Epoch: 6, Loss: 0.4465942680835724, Accuracy: 83.62
Epoch: 7, Loss: 0.2621346414089203, Accuracy: 84.46000000000001
Epoch: 8, Loss: 0.33821406960487366, Accuracy: 85.528
Epoch: 9, Loss: 0.42349013686180115, Accuracy: 86.468
Epoch: 10, Loss: 0.4106876254081726, Accuracy: 87.2


In [12]:
for batch in test_dataloader:
    text, label = batch

    output = transformer(text.to(device))

    loss = criterion(output[:, 0], label.to(device).float())

    correct_predictions += (output[:, 0] > 0.5).eq(label.to(device)).sum().item()
    total_predictions += len(label)

print(f"Loss: {loss.item()}, Accuracy: {correct_predictions / total_predictions * 100}")

Loss: 0.6401382684707642, Accuracy: 71.37400000000001


In [13]:
for batch in test_dataloader:
    text, label = batch
    output = transformer(text.to(device))
    for i in range(0, len(label)):
        print(f"{label[i]} = {output[i]}")
    break


0 = tensor([0.0804], device='cuda:0', grad_fn=<SelectBackward0>)
1 = tensor([0.1118], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.8333], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.0269], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.0905], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.5994], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.0601], device='cuda:0', grad_fn=<SelectBackward0>)
1 = tensor([0.9812], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.7614], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.2487], device='cuda:0', grad_fn=<SelectBackward0>)
1 = tensor([0.9743], device='cuda:0', grad_fn=<SelectBackward0>)
1 = tensor([0.0267], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.2873], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.0298], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.0791], device='cuda:0', grad_fn=<SelectBackward0>)
0 = tensor([0.1406], devi